### Construct the Burrows-Wheeler Transform of a String

In [2]:
def constructBWTstring(text):
    cycles = []

    for i in range(len(text)):
        shift = text[i:] + text[:i]
        cycles.append(shift)

    cycles = sorted(cycles)
    bwt = ''
    for cycle in cycles:
        bwt += cycle[-1]
    print(bwt)

constructBWTstring("GCGTGCCTGGTCA$")

ACTGGCT$TGCGGC


### Reconstruct a String from its Burrows-Wheeler Transform